In [1]:
import numpy as np

from tempo.serve.metadata import ModelFramework, KubernetesOptions
from tempo.serve.model import Model
from tempo.seldon.docker import SeldonDockerRuntime
from tempo.kfserving.protocol import KFServingV2Protocol
from tempo.serve.utils import pipeline, predictmethod
from tempo.seldon.k8s import SeldonKubernetesRuntime
from tempo.serve.utils import pipeline

In [2]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.info("test")

INFO:root:test


In [ ]:
!kubectl create ns production

In [ ]:
!kubectl apply -f ../tempo/tests/testdata/tempo-pipeline-rbac.yaml -n production

In [ ]:
!helm install redis bitnami/redis -n seldon --set usePassword=false

In [5]:
!kaggle datasets download -d uciml/default-of-credit-card-clients-dataset
!unzip -o default-of-credit-card-clients-dataset.zip

100%|██████████████████████████████████████| 0.98M/0.98M [00:00<00:00, 3.41MB/s]
100%|██████████████████████████████████████| 0.98M/0.98M [00:00<00:00, 3.40MB/s]
Archive:  default-of-credit-card-clients-dataset.zip
  inflating: UCI_Credit_Card.csv     


In [3]:
import pandas as pd
data = pd.read_csv('UCI_Credit_Card.csv')

In [4]:
target = 'default.payment.next.month'

In [5]:
import numpy as np
from sklearn.model_selection import train_test_split

OBSERVED_DATA = 15000
TRAIN_1 = 10000
TEST_1 = 5000

REST_DATA = 15000

RUN_DATA = 5000
ROUTE_DATA = 10000

# get features and target
X = data.loc[:, data.columns!=target].values
y = data[target].values

# observed/unobserved split
X_obs, X_rest, y_obs, y_rest = train_test_split(X, y, random_state=1, test_size=REST_DATA)

# observed split into train1/test1
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_obs, y_obs, random_state=1, test_size=TEST_1)

# unobserved split into run/route
X_run, X_route, y_run, y_route = train_test_split(X_rest, y_rest, random_state=1, test_size=ROUTE_DATA)

# observed+run split into train2/test2
X_rest = np.vstack((X_run, X_route))
y_rest = np.hstack((y_run, y_route))

X_train2 = np.vstack((X_train1, X_test1))
X_test2 = X_run

y_train2 = np.hstack((y_train1, y_test1))
y_test2 = y_run

In [17]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=1)
rf.fit(X_train1, y_train1)

RandomForestClassifier(random_state=1)

In [18]:
from xgboost import XGBClassifier
xgb = XGBClassifier(random_state=1)
xgb.fit(X_train2, y_train2)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=1,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [139]:
!mkdir -p mab/sklearn/
!mkdir -p mab/xgboost/

In [140]:
import joblib
joblib.dump(rf, 'mab/sklearn/model.joblib')

['mab/sklearn/model.joblib']

In [141]:
xgb.save_model('mab/xgboost/model.bst')

In [6]:
import os

k8s_options = KubernetesOptions(namespace="production")
k8s_runtime = SeldonKubernetesRuntime(k8s_options=k8s_options)

sklearn_model = Model(
        name="test-iris-sklearn",
        runtime=k8s_runtime,
        platform=ModelFramework.SKLearn,
        uri="gs://seldon-models/mab/sklearn",
        local_folder=os.getcwd()+"/mab/sklearn")

xgboost_model = Model(
        name="test-iris-xgboost",
        runtime=k8s_runtime,
        platform=ModelFramework.XGBoost,
        uri="gs://seldon-models/mab/xgboost",
        local_folder=os.getcwd()+"/mab/xgboost/")

In [ ]:
sklearn_model.upload()
xgboost_model.upload()

In [44]:
k8s_runtime_v2 = SeldonKubernetesRuntime(k8s_options=k8s_options, protocol=KFServingV2Protocol())

@pipeline(name="mab-pipeline",
          runtime=k8s_runtime_v2,
          uri="gs://seldon-models/custom",
          models=[sklearn_model, xgboost_model])
class MABRouter(object):

    def _init(self):
        self.n_branches = 2
        self.beta_params = [1 for _ in range(self.n_branches * 2)]
        
        import logging
        log = logging.getLogger(__name__)
        log.setLevel(10)
        self._log = log
        
        host = "redis-master-0"
        import os
        if os.environ.get("SELDON_LOCAL_ENVIRONMENT"):
            host = "localhost"
            
        self._log.info(f"Setting up redis with host {host}")
            
        import numpy as np
        self._np = np
        
        import redis
        self._rc = redis.Redis(host=host, port=6379)
        self._key = "seldon_deployment_predictor_model_1"
        
        if not self._rc.exists(self._key):
            models_beta_params = [1 for _ in range(self.n_branches * 2)]
            self._rc.lpush(self._key, *models_beta_params)
            
    @predictmethod
    def route(self, payload: np.ndarray) -> np.ndarray:

        if not hasattr(self, "_is_init") or not self._is_init:
            self._init()
            self._is_init = True
        
        models_beta_params = [int(i) for i in self._rc.lrange(self._key, 0, -1)]
        branch_values = [np.random.beta(a, b) for a, b in zip(*[iter(models_beta_params)] * 2)]
        selected_branch = np.argmax(branch_values)
        self._log.info(f"routing to branch: {selected_branch}")
        
        if selected_branch:
            return sklearn_model(payload)
        else:
            return xgboost_model(payload)

In [10]:
%env SELDON_LOCAL_ENVIRONMENT=LOCAL

mab_router = MABRouter()

env: SELDON_LOCAL_ENVIRONMENT=LOCAL


In [ ]:
mab_router.save()

mab_router.upload()

In [50]:
mab_router.deploy()
mab_router.wait_ready()

In [173]:
mab_router.route(payload=X_rest[0:1])

INFO:__main__:routing to branch: 0


array([0.0865844])

In [82]:
@pipeline(name="mab-feedback",
          runtime=k8s_runtime_v2,
          uri="gs://seldon-models/custom")
class MABFeedback(object):

    def _init(self):
        self.n_branches = 2
        self.beta_params = [1 for _ in range(self.n_branches * 2)]
        
        import logging
        log = logging.getLogger(__name__)
        log.setLevel(10)
        self._log = log
        
        host = "redis-master-0"
        import os
        if os.environ.get("SELDON_LOCAL_ENVIRONMENT"):
            host = "localhost"
        
        self._log.info(f"Setting up redis with host {host}")
            
        import numpy as np
        self._np = np
        
        import redis
        self._rc = redis.Redis(host=host, port=6379)
        self._key = "seldon_deployment_predictor_model_1"
        
        if not self._rc.exists(self._key):
            models_beta_params = [1 for _ in range(self.n_branches * 2)]
            self._log.info(f"Creating new key in redis with vals: {models_beta_params}")
            self._rc.lpush(self._key, *models_beta_params)
        else:
            self._log.info("Redis key already exists")
            
    @predictmethod
    def feedback(self, payload: np.ndarray) -> np.ndarray:

        if not hasattr(self, "_is_init") or not self._is_init:
            self._init()
            self._is_init = True
                
        reward = payload[0]
        routing = payload[1]
        truth = payload[2:]

        self._log.info(f"Sending feedback with route {routing} reward {reward} and truth {truth}")
        
        # Currently only support 1 feedback at a time
        n_predictions = 1
        n_success = int(reward * n_predictions)
        n_failures = n_predictions - n_success
    
        self._log.info(f"n_success: {n_success}, n_failures: {n_failures}")

        # Non atomic, race condition op
        self._log.info(f"LINDEX key {self._key} on index {routing*2}")
        success_val = float(self._rc.lindex(self._key, int(routing*2)))
        self._rc.lset(self._key, int(routing*2), str(success_val + n_success))
        fail_val = float(self._rc.lindex(self._key, int(routing*2 + 1)))
        self._rc.lset(self._key, int(routing*2 + 1), str(fail_val + n_failures))

In [80]:
%env SELDON_LOCAL_ENVIRONMENT=LOCAL

mab_feedback = MABFeedback()

env: SELDON_LOCAL_ENVIRONMENT=LOCAL


In [ ]:
mab_feedback.save()

In [ ]:
mab_feedback.upload()

In [ ]:
mab_feedback.deploy()

mab_feedback.wait_ready()

In [81]:
route = 0
reward = 1
mab_feedback.feedback(np.hstack([route, reward, X_train1[0]]))

INFO:__main__:Setting up redis with host localhost
INFO:__main__:Redis key already exists
INFO:__main__:Sending feedback with route 0.0 reward 1.0 and truth [ 1.3020e+04  3.2000e+05  1.0000e+00  2.0000e+00  2.0000e+00  2.8000e+01
  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 -1.0000e+00
  1.1639e+04  7.5080e+03  5.5910e+03  2.3710e+03  4.7540e+03  4.9320e+03
  2.5520e+03  2.0160e+03  0.0000e+00  1.2000e+01  5.0360e+03  1.0306e+04]
INFO:__main__:n_success: 24, n_failures: 0
INFO:__main__:LINDEX key seldon_deployment_predictor_model_1 on index 0.0
